In [2]:
# -*- coding: utf-8 -*-
"""
Created on Mon Apr  4 23:28:32 2022

@author: parkh
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import gym
import collections
import random
from SIM_gannt5 import *

learning_rate = 0.001  
gamma = 0.99
buffer_limit = 10000
batch_size = 16

class ReplayBuffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit);
    def put(self, transition):
        self.buffer.append(transition)
    def sample(self, n):
        mini_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [],[],[],[],[]
        
        for transition in mini_batch:
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])
            
        return torch.tensor(s_lst, dtype=torch. float),torch.tensor(a_lst), torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch. float), torch.tensor(done_mask_lst)
    
    def size(self):
        return len(self.buffer)

class Qnet(nn.Module):
    def __init__(self):
        super(Qnet, self).__init__()
        self.fc1 = nn.Linear(132,64)
        self.fc2 = nn.Linear(64,32)
        self.fc3 = nn.Linear(32,6)
        self.number_of_time_list = np.array([1 for x in range(6)])
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def sample_action(self, obs, epsilon):
        out = self.forward(obs)
        #print(out)
        out2 = out.detach().numpy()
        act_list = out2/self.number_of_time_list
        act = np.argmax(act_list)
        coin = random.random()
        if coin < epsilon:
            return random.randint(0, 5)
        else:
            return act
    def select_action(self, obs, epsilon):
        out = self.forward(obs)
        out2 = out.detach().numpy()
        act_list = out2/self.number_of_time_list
        act = np.argmax(act_list)
        return act,act_list
        
def train(q, q_target, memory, optimizer):
    for i in range(10):
        s,a,r,s_prime,done_mask = memory.sample(batch_size)
        #q.number_of_time_list[a] += 1    
        q_out = q(s)
        q_a = q_out.gather(1,a)
        max_q_prime = q_target(s_prime).max (1)[0].unsqueeze(1)
        target = r + gamma * max_q_prime * done_mask
        loss = F.smooth_l1_loss(q_a, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
            
def main():
    env = FJSP_simulator('C:/Users\parkh/git_tlsgudcks/simulator/data/FJSP_SIM7_all.csv','C:/Users/parkh/FJSP_SETUP_SIM.csv',1)
    q = Qnet()
    q_target = Qnet()
    q_target.load_state_dict(q.state_dict())
    memory = ReplayBuffer()
    print_interval = 1
    q_load = 20
    score = 0.0
    optimizer = optim.Adam(q.parameters(), lr=learning_rate)
    
    for n_epi in range(1000):
        epsilon = max(0.01 , 0.08 - 0.02*(n_epi/200))
        s = env.reset()
        done = False
        score = 0.0
        while not done:
            a = q.sample_action(torch.from_numpy(s). float(), epsilon)
            s_prime, r, done = env.step(a)
            done_mask =0.0 if done else 1.0
            if done == False:
                memory.put((s,a,r,s_prime,done_mask))
                s = s_prime
                score += r
            if done:
                break
        if memory.size()>1000:
            train(q, q_target, memory, optimizer)
            
        if n_epi % print_interval==0 and n_epi!=0:
            #q_target.load_state_dict(q.state_dict())
            Flow_time, machine_utill ,util, makespan = env.performance_measure()
            print("--------------------------------------------------")
            print("flow time: {}, util : {:.3f}, makespan : {}".format(Flow_time, util, makespan))
            print("n_episode: {}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(n_epi, score/print_interval,memory.size(),epsilon*100))
            #score=0.0
        if n_epi % q_load ==0 and n_epi!=0:
            q_target.load_state_dict(q.state_dict())
        s = env.reset()
        done = False
        score = 0.0
    while not done:
        a, a_list = q.select_action(torch.from_numpy(s). float(), epsilon)
        print(a_list)
        print(a)
        s_prime, r, done = env.step(a)
        print(r)
        s = s_prime
        score += r
        if done:
            break
    Flow_time, machine_util, util, makespan = env.performance_measure()
    env.gannt_chart()
    return Flow_time, machine_util, util, makespan, score
Flow_time, machine_util, util, makespan, score =main()
print("FlowTime:" , Flow_time)
print("machine_util:" , machine_util)
print("util:" , util)
print("makespan:" , makespan)
print("Score" , score)
      
    

C:\Users\parkh\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.

<>:91: DeprecationWarning:

invalid escape sequence \p



--------------------------------------------------
flow time: 4719, util : 0.810, makespan : 268
n_episode: 1, score : -1608.0, n_buffer : 226, eps : 8.0%
--------------------------------------------------
flow time: 4687, util : 0.821, makespan : 269
n_episode: 2, score : -1614.0, n_buffer : 342, eps : 8.0%
--------------------------------------------------
flow time: 4651, util : 0.800, makespan : 277
n_episode: 3, score : -1662.0, n_buffer : 457, eps : 8.0%
--------------------------------------------------
flow time: 4869, util : 0.818, makespan : 272
n_episode: 4, score : -1632.0, n_buffer : 572, eps : 8.0%
--------------------------------------------------
flow time: 4822, util : 0.816, makespan : 272
n_episode: 5, score : -1632.0, n_buffer : 687, eps : 8.0%
--------------------------------------------------
flow time: 4785, util : 0.826, makespan : 269
n_episode: 6, score : -1614.0, n_buffer : 802, eps : 7.9%
--------------------------------------------------
flow time: 4850, ut

--------------------------------------------------
flow time: 3585, util : 0.816, makespan : 238
n_episode: 54, score : -1428.0, n_buffer : 6763, eps : 7.5%
--------------------------------------------------
flow time: 3692, util : 0.799, makespan : 233
n_episode: 55, score : -1398.0, n_buffer : 6878, eps : 7.4%
--------------------------------------------------
flow time: 2715, util : 0.803, makespan : 233
n_episode: 56, score : -1398.0, n_buffer : 7010, eps : 7.4%
--------------------------------------------------
flow time: 3453, util : 0.804, makespan : 224
n_episode: 57, score : -1344.0, n_buffer : 7127, eps : 7.4%
--------------------------------------------------
flow time: 3742, util : 0.791, makespan : 225
n_episode: 58, score : -1350.0, n_buffer : 7239, eps : 7.4%
--------------------------------------------------
flow time: 3189, util : 0.729, makespan : 275
n_episode: 59, score : -1650.0, n_buffer : 7367, eps : 7.4%
--------------------------------------------------
flow ti

--------------------------------------------------
flow time: 2670, util : 0.700, makespan : 272
n_episode: 106, score : -1632.0, n_buffer : 10000, eps : 6.9%
--------------------------------------------------
flow time: 2991, util : 0.824, makespan : 238
n_episode: 107, score : -1428.0, n_buffer : 10000, eps : 6.9%
--------------------------------------------------
flow time: 3212, util : 0.806, makespan : 224
n_episode: 108, score : -1344.0, n_buffer : 10000, eps : 6.9%
--------------------------------------------------
flow time: 2903, util : 0.759, makespan : 253
n_episode: 109, score : -1518.0, n_buffer : 10000, eps : 6.9%
--------------------------------------------------
flow time: 2740, util : 0.761, makespan : 246
n_episode: 110, score : -1476.0, n_buffer : 10000, eps : 6.9%
--------------------------------------------------
flow time: 3142, util : 0.806, makespan : 227
n_episode: 111, score : -1362.0, n_buffer : 10000, eps : 6.9%
----------------------------------------------

--------------------------------------------------
flow time: 3761, util : 0.813, makespan : 221
n_episode: 158, score : -1326.0, n_buffer : 10000, eps : 6.4%
--------------------------------------------------
flow time: 2579, util : 0.815, makespan : 221
n_episode: 159, score : -1326.0, n_buffer : 10000, eps : 6.4%
--------------------------------------------------
flow time: 2680, util : 0.741, makespan : 254
n_episode: 160, score : -1524.0, n_buffer : 10000, eps : 6.4%
--------------------------------------------------
flow time: 3466, util : 0.814, makespan : 221
n_episode: 161, score : -1326.0, n_buffer : 10000, eps : 6.4%
--------------------------------------------------
flow time: 3576, util : 0.794, makespan : 218
n_episode: 162, score : -1308.0, n_buffer : 10000, eps : 6.4%
--------------------------------------------------
flow time: 2673, util : 0.814, makespan : 229
n_episode: 163, score : -1374.0, n_buffer : 10000, eps : 6.4%
----------------------------------------------

...

--------------------------------------------------
flow time: 2665, util : 0.793, makespan : 242
n_episode: 886, score : -1452.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 3581, util : 0.792, makespan : 213
n_episode: 887, score : -1278.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 2720, util : 0.694, makespan : 279
n_episode: 888, score : -1674.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 2636, util : 0.801, makespan : 233
n_episode: 889, score : -1398.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 2631, util : 0.807, makespan : 231
n_episode: 890, score : -1386.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 3531, util : 0.815, makespan : 218
n_episode: 891, score : -1308.0, n_buffer : 10000, eps : 1.0%
----------------------------------------------

--------------------------------------------------
flow time: 2717, util : 0.649, makespan : 293
n_episode: 938, score : -1758.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 3087, util : 0.790, makespan : 231
n_episode: 939, score : -1386.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 2725, util : 0.728, makespan : 276
n_episode: 940, score : -1656.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 3167, util : 0.811, makespan : 213
n_episode: 941, score : -1278.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 3319, util : 0.822, makespan : 228
n_episode: 942, score : -1368.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 3510, util : 0.800, makespan : 213
n_episode: 943, score : -1278.0, n_buffer : 10000, eps : 1.0%
----------------------------------------------

--------------------------------------------------
flow time: 3610, util : 0.790, makespan : 219
n_episode: 990, score : -1314.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 3455, util : 0.777, makespan : 220
n_episode: 991, score : -1320.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 3525, util : 0.809, makespan : 211
n_episode: 992, score : -1266.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 3555, util : 0.792, makespan : 219
n_episode: 993, score : -1314.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 3491, util : 0.786, makespan : 213
n_episode: 994, score : -1278.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 2715, util : 0.652, makespan : 294
n_episode: 995, score : -1764.0, n_buffer : 10000, eps : 1.0%
----------------------------------------------

-10
[-414.72711182 -426.48773193 -425.6817627  -414.81265259 -416.75170898
 -429.47927856]
0
-8
[-414.73406982 -426.1434021  -425.73391724 -414.6652832  -416.62661743
 -429.42868042]
3
-16
[-405.25726318 -415.87936401 -415.27682495 -405.31182861 -406.96710205
 -418.722229  ]
0
-13
[-401.96524048 -412.66021729 -412.17211914 -402.23318481 -403.89599609
 -415.59829712]
0
-16
[-393.38009644 -403.31942749 -402.81610107 -393.64770508 -395.01754761
 -406.07046509]
0
-16
[-386.86773682 -396.41732788 -395.80233765 -387.27810669 -388.46640015
 -398.88116455]
0
-15
[-376.53872681 -385.59024048 -385.36605835 -377.04782104 -378.2381897
 -388.15725708]
0
-12
[-374.10421753 -383.29885864 -383.44216919 -374.68161011 -375.94329834
 -386.16305542]
0
-16
[-370.05462646 -379.36901855 -379.48321533 -370.70117188 -372.01419067
 -382.42752075]
0
-11
[-359.93469238 -368.56869507 -368.80911255 -360.62634277 -361.80908203
 -371.45278931]
0
-18
[-348.90087891 -357.31161499 -357.99264526 -349.55792236 -350.863830

FlowTime: 3280
machine_util: 0
util: 0.7489539748953975
makespan: 239
Score -1434.0
